In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout,Dense,Flatten,BatchNormalization,Activation,Input
from keras.layers.convolutional import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split, StratifiedKFold
from keras.utils.vis_utils import plot_model
from keras.utils import np_utils
from matplotlib import pyplot
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, LearningRateScheduler, TensorBoard, EarlyStopping, CSVLogger
from keras import backend as K

Using TensorFlow backend.


In [2]:
def cnn_model():
    model = Sequential()
    model.add(Conv2D(32,(7,7), input_shape=(224,224,3), kernel_initializer='he_uniform',activation='relu'))
    model.add(BatchNormalization())
    
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    
    model.add(Conv2D(32,(5,5), kernel_initializer='he_uniform',activation='relu'))
    model.add(BatchNormalization())
    
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    
    model.add(Conv2D(32,(3,3), kernel_initializer='he_uniform',activation='relu'))
    model.add(BatchNormalization())
    
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    
    model.add(Flatten())
    
    model.add(Dense(128, activation='selu',kernel_initializer='he_normal'))
    model.add(Dropout(0.5))
    model.add(Dense(10,activation='selu',kernel_initializer='he_normal'))
    model.add(Dense(2,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

In [3]:
train_datagen = ImageDataGenerator(rotation_range=180,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   zoom_range = 0.25)

test_datagen = ImageDataGenerator(rotation_range=None,
                                   horizontal_flip=True,
                                   vertical_flip=True)

In [4]:
path = 'train'
train_generator = train_datagen.flow_from_directory(path,
                                                    target_size=(224, 224),
                                                    batch_size=64,
                                                    class_mode='categorical')
path = 'test'
test_generator = test_datagen.flow_from_directory(path,
                                                    target_size=(224, 224),
                                                    batch_size=64,
                                                    class_mode='categorical')

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [5]:
cnn_model = cnn_model()
cnn_model.summary()
plot_model(cnn_model, to_file='cnn.png')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 218, 218, 32)      4736      
_________________________________________________________________
batch_normalization_1 (Batch (None, 218, 218, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 109, 109, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 105, 105, 32)      25632     
_________________________________________________________________
batch_normalization_2 (Batch (None, 105, 105, 32)      128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 53, 53, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 51, 51, 32)        9248      
__________

In [ ]:
batch_size = 32
epochs = 100
class_weight = {0:1,1:4.347}
history = cnn_model.fit_generator(
        train_generator,
        steps_per_epoch = 50 ,
        class_weight = class_weight,
        epochs = epochs,
        validation_data=test_generator,
        validation_steps = 13)

Epoch 1/10
50/50 [==============================] - 320s - loss: 0.4988 - acc: 0.7016 - val_loss: 0.5712 - val_acc: 0.8166
Epoch 2/10
50/50 [==============================] - 313s - loss: 0.4483 - acc: 0.8125 - val_loss: 0.5242 - val_acc: 0.8071
Epoch 3/10
50/50 [==============================] - 313s - loss: 0.4356 - acc: 0.8131 - val_loss: 0.5080 - val_acc: 0.8139
Epoch 4/10
50/50 [==============================] - 313s - loss: 0.4291 - acc: 0.8147 - val_loss: 0.4958 - val_acc: 0.8125
Epoch 5/10
50/50 [==============================] - 312s - loss: 0.4385 - acc: 0.8100 - val_loss: 0.5114 - val_acc: 0.8084
Epoch 6/10
 3/50 [>.............................] - ETA: 271s - loss: 0.4581 - acc: 0.8073

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'],color='r')
plt.plot(history.history['val_acc'],color='g')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'],color='r')
plt.plot(history.history['val_loss'],color='g')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()